In [ ]:
pip show azure-ai-ml

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
ml_client = MLClient.from_config(credential=credential)

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key"
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model"
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json"
)

if response[1] == '1':
    print("Diabetic")
else:
    print("Not diabetic")

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint.traffic)

print(endpoint.scoring_uri)

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)